<a href="https://colab.research.google.com/github/aboelela924/Deep-Reinforcement-Learning-Hands-On/blob/master/Deep_Reinforcement_Learning_Hands_On_Chapter_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install tensorboardX

     |████████████████████████████████| 317kB 3.5MB/s 


In [5]:
import gym 
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch 
import torch.nn as nn 
import torch.optim as optim

In [6]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [9]:
class Net(nn.Module):
    def __init__(self, obervtion_size, hidden_size, number_of_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obervtion_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, number_of_actions)
        )

    def call(self, x):
        return self.net(x)

In [10]:
Episode = namedtuple("Episode", field_names=["reward", "steps"])
EpisodeStep = namedtuple("EpisodeStep", field_names=["observation", "action"])

In [25]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    softmax = nn.Softmax(dim=1)
    
    while True:
        obs_tensor = torch.FloatTensor([obs])
        action_probabilites = net(obs_tensor)
        action_probabilites = action_probabilites.data.numpy()[0]
        action = np.random.choice(len(action_probabilites), p=action_probabilites)
        new_state, reward, is_done, info = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=new_state, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            obs = env.reset()
            if(len(batch) == batch_size):
                yield batch
                batch = []
        obs = new_state

In [13]:
def filter(batch, percentile):
    rewards = list(map(lambda e: e.reward, batch))
    rewards_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_observations = []
    train_actions = []

    for episode in batch:
        if episode.reward < rewards_bound:
            continue
        train_observations.extend(map(lambda episode_step: episode_step.observation , episode.steps))
        train_actions.extend(map(lambda episode_step: episode_step.action , episode.steps))
    
    return torch.FloatTensor(train_observations), torch.FloatTensor(train_actions), rewards_bound, reward_mean

In [ ]:
env = gym.make("CartPole-v0")

observation_size = env.observation_space.shape[0]
actions_size = env.action_space.n

net = Net(observation_size, HIDDEN_SIZE, actions_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

for i, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    train_observations, train_actions, rewards_bound, reward_mean = filter(batch, PERCENTILE)
    predicted_actions = net(train_observations)
    optimizer.zero_grad()
    loss = criterion(train_actions, predicted_actions)
    loss.backwards()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (i, loss.item(), reward_mean, rewards_bound))
    if reward_mean > 199:
        print("Solved!")
        break